In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15787460701246559961, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 913595652912368180
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:83:00.0, compute capability: 6.1", name: "/device:GPU:1"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 4198446168895324107
 physical_device_desc: "device: 1, name: GeForce GTX 1080 Ti, pci bus id: 0000:84:00.0, compute capability: 6.1", name: "/device:GPU:2"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 12372457403774952053
 physical_device_desc: "device: 2, name: GeForce GTX 1080 Ti, pci bus id: 0000:87:00.0, compute capability: 6.1", name: "/device:GPU:3"
 device_type: "GPU"
 memory_limit: 10968950375
 locality {
   bus_id: 2
 }
 incarnation: 1104659660096804096
 physical_devic

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time

import os
import pandas as pd

import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize

import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.4.0


In [5]:
output_classes = 2
batch_size = 64
img_height, img_width = 224, 224
input_shape = (img_height, img_width, 3)
epochs = 50

nb_train_samples = 103104
nb_validation_samples = 12288
nb_test_samples = 697

In [6]:
train_dir = 'data/train/'
validation_dir = 'data/validation'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = True,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 103104 images belonging to 2 classes.
Found 12288 images belonging to 2 classes.
Found 697 images belonging to 2 classes.


In [8]:
def vgg19_model():
    base_model = VGG19(weights='imagenet', include_top=False, pooling='avg')
    x = base_model.output
    # x = GlobalAveragePooling2D()(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    x = Dense(1024, kernel_regularizer=l2(0.0001), bias_regularizer=l2(0.0001))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.5)(x)
    prediction = Dense(output_classes, activation=tf.nn.softmax)(x)

    model = Model(inputs=base_model.input,outputs=prediction)
    return model

In [9]:
space = {
         'lr': hp.choice('lr',[0.001, 0.0001, 0.00001,0.000001]),
         'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
         'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.3,0.4,0.5,0.6,0.7,0.8]),
         'beta_2':hp.choice('beta_2',[0.99,0.995,0.7,0.8,0.9,0.999]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
         'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [14]:
def f_nn(params):   
    print ('Parameters testing: ', params)
#     batch_size=params['batch_size']

#     random_seed = np.random.seed(1132)

#     train_datagen = ImageDataGenerator(
#         rescale=1. / 255,
#         featurewise_center=True,
#         featurewise_std_normalization=True)

#     test_datagen = ImageDataGenerator(rescale=1. / 255)

#     train_generator = train_datagen.flow_from_directory(
#         train_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     validation_generator = train_datagen.flow_from_directory(
#         validation_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = True,
#         seed = random_seed,
#         class_mode='categorical')

#     test_generator = test_datagen.flow_from_directory(
#         test_dir,
#         target_size=(img_height, img_width),
#         batch_size=batch_size,
#         shuffle = False,
#         class_mode='categorical')

   
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0, amsgrad=params['amsgrad'])
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = vgg19_model()
    
    model.compile(optimizer = adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit_generator(
      train_generator,
      steps_per_epoch = nb_train_samples // batch_size,
      epochs = 15,
      validation_data = validation_generator,
      validation_steps = nb_validation_samples // batch_size,
      verbose = 1)
    
    score = model.evaluate_generator(validation_generator, steps=50)

    print ('Validation Score: ', score[0])
    print ('Validation Accuracy: ',score[1])
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    predicts = model.predict_generator(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
    predict_class = np.argmax(predicts, axis=1)
    errors = np.where(predict_class != truth)[0]
    print("No of errors = {}/{}".format(len(errors),test_generator.samples))
    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': score[0], 'status': STATUS_OK, 'model': model}

In [15]:
# def get_callbacks(params):
#     callbacks =[EarlyStopping(monitor='val_acc', patience=5, verbose=1),
#                 ModelCheckpoint('callbacks/{}.h5'.format(params['batch_size']), save_best_only=True),
#              TensorBoard('tensorlogs/logs-gridsearch', write_graph=True, write_grads=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)]
#     return callbacks

In [16]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals=30, trials=trials)
print(best)

Parameters testing:  {'amsgrad': False, 'beta_1': 0.7, 'beta_2': 0.995, 'dropout': 0.6, 'lr': 0.0001, 'optimizer': 'adam'}
Epoch 1/15
1611/1611 [==============================] - 1215s 754ms/step - loss: 0.4091 - acc: 0.8957 - val_loss: 0.3003 - val_acc: 0.9296
Epoch 2/15
1611/1611 [==============================] - 1209s 750ms/step - loss: 0.2012 - acc: 0.9586 - val_loss: 0.3521 - val_acc: 0.9028
Epoch 3/15
1611/1611 [==============================] - 1208s 750ms/step - loss: 0.1100 - acc: 0.9800 - val_loss: 0.1507 - val_acc: 0.9543
Epoch 4/15
1611/1611 [==============================] - 1201s 745ms/step - loss: 0.0696 - acc: 0.9874 - val_loss: 0.1397 - val_acc: 0.9577
Epoch 5/15
1611/1611 [==============================] - 1208s 750ms/step - loss: 0.0478 - acc: 0.9908 - val_loss: 0.0865 - val_acc: 0.9733
Epoch 6/15
1611/1611 [==============================] - 1219s 757ms/step - loss: 0.0352 - acc: 0.9929 - val_loss: 0.1086 - val_acc: 0.9650
Epoch 7/15
1611/1611 [=====================

Epoch 5/15
1611/1611 [==============================] - 1224s 760ms/step - loss: 0.0484 - acc: 0.9910 - val_loss: 0.2093 - val_acc: 0.9510
Epoch 6/15
1611/1611 [==============================] - 1213s 753ms/step - loss: 0.0367 - acc: 0.9924 - val_loss: 0.1016 - val_acc: 0.9713
Epoch 7/15
1611/1611 [==============================] - 1209s 750ms/step - loss: 0.0278 - acc: 0.9939 - val_loss: 0.8224 - val_acc: 0.8446
Epoch 8/15
1611/1611 [==============================] - 1210s 751ms/step - loss: 0.0228 - acc: 0.9947 - val_loss: 0.2658 - val_acc: 0.9503
Epoch 9/15
1611/1611 [==============================] - 1209s 750ms/step - loss: 0.0227 - acc: 0.9946 - val_loss: 0.1002 - val_acc: 0.9758
Epoch 10/15
1611/1611 [==============================] - 1207s 749ms/step - loss: 0.0192 - acc: 0.9952 - val_loss: 0.3277 - val_acc: 0.9420
Epoch 11/15
1611/1611 [==============================] - 1208s 750ms/step - loss: 0.0164 - acc: 0.9962 - val_loss: 0.3812 - val_acc: 0.9230
Epoch 12/15
1611/1611 [==

Epoch 10/15
1611/1611 [==============================] - 1222s 759ms/step - loss: 0.1653 - acc: 0.9984 - val_loss: 0.2327 - val_acc: 0.9757
Epoch 11/15
1611/1611 [==============================] - 1226s 761ms/step - loss: 0.1636 - acc: 0.9986 - val_loss: 0.2364 - val_acc: 0.9752
Epoch 12/15
1611/1611 [==============================] - 1234s 766ms/step - loss: 0.1626 - acc: 0.9989 - val_loss: 0.2215 - val_acc: 0.9809
Epoch 13/15
1611/1611 [==============================] - 1222s 759ms/step - loss: 0.1607 - acc: 0.9993 - val_loss: 0.2247 - val_acc: 0.9801
Epoch 14/15
1611/1611 [==============================] - 1234s 766ms/step - loss: 0.1600 - acc: 0.9993 - val_loss: 0.2317 - val_acc: 0.9753
Epoch 15/15
1611/1611 [==============================] - 1231s 764ms/step - loss: 0.1589 - acc: 0.9994 - val_loss: 0.2126 - val_acc: 0.9828
Validation Score:  0.18765003800392152
Validation Accuracy:  0.9890625
11/10 [==============================] - 10s 865ms/step
No of errors = 14/697
*_**_**_**_

ResourceExhaustedError: OOM when allocating tensor with shape[64,64,224,224]
	 [[Node: training_9/Adam/gradients/block1_pool_10/MaxPool_grad/MaxPoolGrad = MaxPoolGrad[T=DT_FLOAT, _class=["loc:@block1_pool_10/MaxPool"], data_format="NHWC", ksize=[1, 2, 2, 1], padding="VALID", strides=[1, 2, 2, 1], _device="/job:localhost/replica:0/task:0/device:GPU:0"](block1_conv2_10/Relu, block1_pool_10/MaxPool, training_9/Adam/gradients/block2_conv1_10/convolution_grad/Conv2DBackpropInput)]]

Caused by op 'training_9/Adam/gradients/block1_pool_10/MaxPool_grad/MaxPoolGrad', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Data/home/sah623/venv/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Data/home/sah623/venv/lib/python3.6/site-packages/traitlets/config/application.py", line 658, i